In [1]:
import numpy as np
import pandas as pd


In [3]:
acc_df = pd.read_pickle('/shared/accessible_pois_lon_lat_wheelchair.pkl')
acc_lonlat=acc_df[['lat','lon']].to_numpy()

In [4]:
acconly_df=acc_df.loc[acc_df['wheelchair_cat'] == 'yes']
acconly_df_lonlat=acconly_df[['lat','lon']].to_numpy()

## Distance Calculation Based on Latitude and Longitude

reference: https://www.kite.com/python/answers/how-to-find-the-distance-between-two-lat-long-coordinates-in-python

In [5]:
def distance_search (POI, otherPOI, meternearby):
    lat1 = np.radians(POI[0,1])
    lon1 = np.radians(POI[0,0])
    lat2 = np.radians(otherPOI[:,1])
    lon2 = np.radians(otherPOI[:,0])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    R = 6373.0
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan(np.sqrt(a)/ np.sqrt(1 - a))
    distance = R * c*1000
    distance_sorted=np.sort(distance,axis=None)
    count=0
    for i in distance_sorted:
        if i < meternearby:
            count = count + 1
        else:
            break
    return count

In [ ]:
numOfPoiNearby_100m=np.zeros([1,len(acc_lonlat)])
for i in range (len(acc_lonlat)):
    numOfPoiNearby_100m[0,i]=distance_search(acc_lonlat[i].reshape((1,2)),acc_lonlat,100)

In [ ]:
numOfaccessiblePoiNearby_100m=np.zeros([1,len(acc_lonlat)])
for i in range (len(acc_lonlat)):
    numOfaccessiblePoiNearby_100m[0,i]=distance_search(acc_lonlat[i].reshape((1,2)),acconly_df_lonlat,100)

In [ ]:
numOfPoiNearby_1000m=np.zeros([1,len(acc_lonlat)])
for i in range (len(acc_lonlat)):
    numOfPoiNearby_1000m[0,i]=distance_search(acc_lonlat[i].reshape((1,2)),acc_lonlat,1000)

In [ ]:
numOfaccessiblePoiNearby_1000m=np.zeros([1,len(acc_lonlat)])
for i in range (len(acc_lonlat)):
    numOfaccessiblePoiNearby_1000m[0,i]=distance_search(acc_lonlat[i].reshape((1,2)),acconly_df_lonlat,1000)

In [ ]:
sLength = len(acc_df['lon'])

In [ ]:
acc_df['numOfPoiNearby_100m'] = pd.Series(numOfPoiNearby_100m.reshape(sLength,), index=acc_df.index)
acc_df['numOfaccessiblePoiNearby_100m'] = pd.Series(numOfaccessiblePoiNearby_100m.reshape(sLength,), index=acc_df.index)

In [ ]:
acc_df['numOfPoiNearby_1000m'] = pd.Series(numOfPoiNearby_1000m.reshape(sLength,), index=acc_df.index)
acc_df['numOfaccessiblePoiNearby_1000m'] = pd.Series(numOfaccessiblePoiNearby_1000m.reshape(sLength,), index=acc_df.index)

In [16]:
acc_df

,lon,lat,wheelchair_cat,numOfPoiNearby_100m,numOfaccessiblePoiNearby_100m,numOfPoiNearby_1000m,numOfaccessiblePoiNearby_1000m
99878,-0.152985,51.524358,yes,2.0,2.0,98.0,48.0
104734,-1.785876,51.565653,yes,1.0,1.0,8.0,8.0
106213,-0.142942,51.525660,yes,3.0,3.0,107.0,57.0
108042,-0.135513,51.523561,limited,3.0,1.0,362.0,184.0
251191,-0.715918,51.115444,limited,2.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...
8495396114,-2.997905,51.589057,yes,6.0,6.0,42.0,40.0
8495396115,-2.997255,51.589114,yes,7.0,7.0,47.0,45.0
8497470120,-0.135463,51.602958,limited,1.0,0.0,2.0,1.0
8497671325,-4.336600,55.905952,no,1.0,0.0,2.0,0.0


In [17]:
import pickle
acc_df.to_pickle('/shared/numberOfNearbyPOIs.pkl')